# MSFS (Multi-agent Smart Factory Scheduling) Environment Tutorial

This notebook provides a tutorial for the MSFS environment where agents manage jobs and machines in a smart factory scheduling scenario.

## Overview

The MSFS environment simulates a smart factory where:
- **Jobs** arrive with specific processing requirements
- **Machines** with different capabilities process jobs
- **Agents** coordinate scheduling and resource allocation
- **Optimization goals** include throughput, wait times, and resource utilization

In [1]:
import sys
import os
sys.path.append(os.path.join('..'))

import numpy as np
from MSFS.env_msfs import create_msfs_env
from MSFS.env_msfs_ctde import create_msfs_ctde_env

## Environment Setup

### Core Components:
- **Jobs**: Tasks with processing requirements and deadlines
- **Machines**: Production resources with different capabilities
- **Agents**: Scheduling and coordination managers
- **Production Lines**: Sequential processing workflows

In [4]:
# Create environment
env = create_msfs_env(difficulty="normal", render_mode="")

print("MSFS Environment Info:")
env_info = env.get_env_info()
for key, value in env_info.items():
    print(f"  {key}: {value}")


MSFS Environment Info:
  n_agents: 2
  agent_ids: ['agent_0', 'agent_1']
  action_space: Discrete(8)
  observation_space: Box(-1.0, 1.0, (24,), float32)
  global_state_dim: 42
  max_steps: 50
  episode_limit: 50
  obs_shape: 24
  n_actions: 8
  state_shape: 42


## Action and Observation Spaces

In [5]:
# Reset and examine spaces
obs = env.reset()

print("Action Space:")
print(f"  Actions: Job assignment, machine scheduling, resource allocation")
print(f"  N actions per agent: {env_info['n_actions']}")

print("\nObservation Space:")
for agent_id, observation in obs.items():
    print(f"  {agent_id}: shape={observation.shape}, range=[{observation.min():.3f}, {observation.max():.3f}]")
    
    if agent_id == list(obs.keys())[0]:
        print(f"    Contains: job_queue, machine_status, processing_times, etc.")

Action Space:
  Actions: Job assignment, machine scheduling, resource allocation
  N actions per agent: 8

Observation Space:
  agent_0: shape=(24,), range=[-1.000, 1.000]
    Contains: job_queue, machine_status, processing_times, etc.
  agent_1: shape=(24,), range=[-1.000, 1.000]


## Reward System

### Reward Components:
- **Job completion**: Positive reward for processed jobs
- **On-time delivery**: Bonus for meeting deadlines
- **Resource utilization**: Rewards for efficient machine usage
- **Wait time penalties**: Negative rewards for job delays
- **Throughput rewards**: Higher rewards for higher production rates

In [6]:
print("Reward System:")
print("  Positive rewards:")
print("    - Job completion (+base reward)")
print("    - On-time delivery (+bonus)")
print("    - High throughput (+bonus)")
print("    - Resource utilization (+bonus)")
print("  Negative rewards:")
print("    - Job delays (-penalty)")
print("    - Machine idle time (-penalty)")
print("    - Missed deadlines (-large penalty)")

print("\nConfiguration Examples:")
print("  Easy: Relaxed deadlines, high rewards")
print("  Normal: Balanced constraints")
print("  Hard: Tight deadlines, high penalties")

Reward System:
  Positive rewards:
    - Job completion (+base reward)
    - On-time delivery (+bonus)
    - High throughput (+bonus)
    - Resource utilization (+bonus)
  Negative rewards:
    - Job delays (-penalty)
    - Machine idle time (-penalty)
    - Missed deadlines (-large penalty)

Configuration Examples:
  Easy: Relaxed deadlines, high rewards
  Normal: Balanced constraints
  Hard: Tight deadlines, high penalties


## CTDE Integration

In [13]:
# Create CTDE environment
ctde_env = create_msfs_ctde_env(difficulty="normal", global_state_type="concat")

obs = ctde_env.reset()
global_state = ctde_env.get_global_state()


ctde_env.close()

## Key Features

### 1. Complex Scheduling Problem
- Multiple job types with different requirements
- Machine constraints and capabilities
- Sequential and parallel processing workflows
- Dynamic job arrival patterns

### 2. Multi-objective Optimization
- Maximize throughput
- Minimize wait times
- Optimize resource utilization
- Meet deadlines and quality requirements

### 3. Real-world Applications
- Manufacturing scheduling
- Supply chain optimization
- Resource allocation planning
- Production line management

In [14]:
# Clean up
env.close()
print("\nMSFS Environment Tutorial completed!")
print("\nKey takeaways:")
print("  1. Complex multi-objective optimization problem")
print("  2. Real-world factory scheduling simulation")
print("  3. Coordination required for optimal performance")
print("  4. Rich reward system for multiple optimization goals")


MSFS Environment Tutorial completed!

Key takeaways:
  1. Complex multi-objective optimization problem
  2. Real-world factory scheduling simulation
  3. Coordination required for optimal performance
  4. Rich reward system for multiple optimization goals
